In [1]:
#%pip install langchain
#%pip install "langserve[all]"
#%pip install langchain-cli
#%pip install langchain-openai
#%pip install python-dotenv
#%pip install openai
#%pip install unstructured
#%pip install faiss-cpu
#%pip install chromadb
#%pip install langchainhub
#%pip install pypdf

In [2]:
import openai_setup

AZURE_OPENAI_ENDPOINT:https://anildwaopenaiwestus.openai.azure.com/


In [80]:
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import SystemMessagePromptTemplate
import os

# Initiate a connection to the LLM from Azure OpenAI Service via LangChain.
llm = AzureChatOpenAI(

    deployment_name=os.environ["DEPLOYMENT_NAME"]
)

In [81]:
from langchain.schema.messages import HumanMessage

message = HumanMessage(
    content="Translate this sentence from English to French. I love programming."
)
llm([message])

c:\Users\anildwa\AppData\Local\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


AIMessage(content="J'aime programmer.")

In [11]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/PayPal SDD - Migration Example no JSDK.pdf")
pdf_pages = loader.load_and_split()

In [12]:
len(pdf_pages)

22

In [13]:
from langchain_community.document_loaders import UnstructuredURLLoader 

urls  = ["https://developer.paypal.com/api/nvp-soap/NVPAPIOverview/",
         "https://developer.paypal.com/api/nvp-soap/get-started/",
         "https://developer.paypal.com/api/nvp-soap/PayPalSOAPAPIArchitecture/",
         "https://developer.paypal.com/api/nvp-soap/apiCredentials/",
         "https://developer.paypal.com/api/nvp-soap/nvpsoap-sdks/",
         "https://developer.paypal.com/api/nvp-soap/set-express-checkout-nvp/",
         "https://developer.paypal.com/api/nvp-soap/set-express-checkout-soap/",
         "https://developer.paypal.com/api/nvp-soap/endpoints/",
         ]
loader = UnstructuredURLLoader(urls=urls)
data = loader.load()
data

[Document(page_content='NVP/SOAP API /\n\nGet Started /\n\nNVP Basics\n\nPayPal NVP API Basics\n\nAPILegacy\n\nThe Name-Value Pair (NVP) API provides parameter-based association between the request and response fields of a message and their values. The request message is sent from your website by the API, and a response message is returned by PayPal using a client-server model in which your site is a client of the PayPal server. NVP describes the format of the request and response messages that you send to and receive from the PayPal API. An example of a name-value pair is <FieldName>=<Value>, where <FieldName> is the name of the PayPal API field and <Value> is the information that you pass to or receive from the PayPal API, such as, VERSION=200.\n\nNote: The PayFlow API also uses name-value pairs to provide parameter-based association between request and response fields of a message and their values; however, the PayFlow API is not the same as the NVP API; for more information about t

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=300,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

page_contents = []
for d in data:
    
    print(d.metadata)
    texts = text_splitter.create_documents([d.page_content])
    for text in texts:
        text.metadata = d.metadata
        page_contents.append(text)
    #page_contents.append(text.page_content for text in texts)

for text in page_contents:
    print(text.metadata)


for p in pdf_pages:
    print(p.metadata)
    texts = text_splitter.create_documents([p.page_content])
    for text in texts:
        text.metadata = d.metadata
        page_contents.append(text)

Created a chunk of size 1431, which is longer than the specified 1000
Created a chunk of size 1172, which is longer than the specified 1000
Created a chunk of size 1232, which is longer than the specified 1000
Created a chunk of size 8055, which is longer than the specified 1000
Created a chunk of size 1324, which is longer than the specified 1000
Created a chunk of size 9014, which is longer than the specified 1000
Created a chunk of size 6614, which is longer than the specified 1000
Created a chunk of size 1732, which is longer than the specified 1000
Created a chunk of size 1032, which is longer than the specified 1000
Created a chunk of size 2232, which is longer than the specified 1000
Created a chunk of size 5449, which is longer than the specified 1000
Created a chunk of size 8152, which is longer than the specified 1000
Created a chunk of size 1211, which is longer than the specified 1000
Created a chunk of size 8456, which is longer than the specified 1000
Created a chunk of s

{'source': 'https://developer.paypal.com/api/nvp-soap/NVPAPIOverview/'}
{'source': 'https://developer.paypal.com/api/nvp-soap/get-started/'}
{'source': 'https://developer.paypal.com/api/nvp-soap/PayPalSOAPAPIArchitecture/'}
{'source': 'https://developer.paypal.com/api/nvp-soap/apiCredentials/'}
{'source': 'https://developer.paypal.com/api/nvp-soap/nvpsoap-sdks/'}
{'source': 'https://developer.paypal.com/api/nvp-soap/set-express-checkout-nvp/'}
{'source': 'https://developer.paypal.com/api/nvp-soap/set-express-checkout-soap/'}
{'source': 'https://developer.paypal.com/api/nvp-soap/endpoints/'}
{'source': 'https://developer.paypal.com/api/nvp-soap/NVPAPIOverview/'}
{'source': 'https://developer.paypal.com/api/nvp-soap/NVPAPIOverview/'}
{'source': 'https://developer.paypal.com/api/nvp-soap/NVPAPIOverview/'}
{'source': 'https://developer.paypal.com/api/nvp-soap/NVPAPIOverview/'}
{'source': 'https://developer.paypal.com/api/nvp-soap/NVPAPIOverview/'}
{'source': 'https://developer.paypal.com/a

In [16]:

len(page_contents)

127

In [18]:
import os
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings


embeddings_from_model = AzureOpenAIEmbeddings(azure_deployment=os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"])
#embeddings = embeddings_model.embed_documents(page_contents)

store = LocalFileStore("./cache/")

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embeddings_from_model, store, namespace=embeddings_from_model.model
)

In [19]:
list(store.yield_keys())

[]

In [20]:
%%time
try: 
    faiss_db = FAISS.load_local("faiss_index", cached_embedder)
except(Exception) as e:
    print("Local index not found, creating new one")   
    faiss_db = FAISS.from_documents(page_contents, cached_embedder)
    faiss_db.save_local("faiss_index")

Local index not found, creating new one
CPU times: total: 1.56 s
Wall time: 26.5 s


In [21]:
len(list(store.yield_keys()))

126

In [22]:
query = "how to use paypal api as client server model"
docs = faiss_db.similarity_search_with_score(query, k=5)

print(len(docs))
for doc, score in docs:
    print(f"Content: {doc.page_content},\n\nScore: {score}")
    print("--------------------------")

5
Content: Note: The PayFlow API also uses name-value pairs to provide parameter-based association between request and response fields of a message and their values; however, the PayFlow API is not the same as the NVP API; for more information about the PayFlow API, see the Payflow Gateway Developer Guide and Reference.

PayPal API client-server architecture

The PayPal API uses a client-server model. Your website is a client of the PayPal server.

A page on your website initiates an action on a PayPal API server by sending a request to the server. The PayPal server responds with a confirmation that the requested action was taken or indicates that an error occurred. The response might also contain additional information related to the request. The following diagram shows the basic request-response mechanism.,

Score: 0.3118332028388977
--------------------------
Content: Like many web services, PayPal SOAP is a combination of client-side and server-side schemas, hardware and software s

In [23]:
from langchain_community.vectorstores import Chroma
chroma_db = Chroma.from_documents(page_contents, AzureOpenAIEmbeddings(), persist_directory="./chroma")

In [24]:
query = "how to use paypal api as client server model"

embedding_vector = AzureOpenAIEmbeddings().embed_query(query)
docs = chroma_db.similarity_search_by_vector(embedding_vector, k=5)
print(docs[0].page_content)

Note: The PayFlow API also uses name-value pairs to provide parameter-based association between request and response fields of a message and their values; however, the PayFlow API is not the same as the NVP API; for more information about the PayFlow API, see the Payflow Gateway Developer Guide and Reference.

PayPal API client-server architecture

The PayPal API uses a client-server model. Your website is a client of the PayPal server.

A page on your website initiates an action on a PayPal API server by sending a request to the server. The PayPal server responds with a confirmation that the requested action was taken or indicates that an error occurred. The response might also contain additional information related to the request. The following diagram shows the basic request-response mechanism.


In [25]:

docs = chroma_db.similarity_search_with_score(query, k=5)

print(len(docs))
for doc, score in docs:
    print(f"Content: {doc.page_content},\n\nScore: {score}")
    print("--------------------------")

5
Content: Note: The PayFlow API also uses name-value pairs to provide parameter-based association between request and response fields of a message and their values; however, the PayFlow API is not the same as the NVP API; for more information about the PayFlow API, see the Payflow Gateway Developer Guide and Reference.

PayPal API client-server architecture

The PayPal API uses a client-server model. Your website is a client of the PayPal server.

A page on your website initiates an action on a PayPal API server by sending a request to the server. The PayPal server responds with a confirmation that the requested action was taken or indicates that an error occurred. The response might also contain additional information related to the request. The following diagram shows the basic request-response mechanism.,

Score: 0.3118331730365753
--------------------------
Content: Like many web services, PayPal SOAP is a combination of client-side and server-side schemas, hardware and software s

In [26]:
faiss_retriever = faiss_db.as_retriever()
docs = faiss_retriever.get_relevant_documents(query)


In [27]:
docs

[Document(page_content='Note: The PayFlow API also uses name-value pairs to provide parameter-based association between request and response fields of a message and their values; however, the PayFlow API is not the same as the NVP API; for more information about the PayFlow API, see the Payflow Gateway Developer Guide and Reference.\n\nPayPal API client-server architecture\n\nThe PayPal API uses a client-server model. Your website is a client of the PayPal server.\n\nA page on your website initiates an action on a PayPal API server by sending a request to the server. The PayPal server responds with a confirmation that the requested action was taken or indicates that an error occurred. The response might also contain additional information related to the request. The following diagram shows the basic request-response mechanism.', metadata={'source': 'https://developer.paypal.com/api/nvp-soap/NVPAPIOverview/'}),
 Document(page_content="Like many web services, PayPal SOAP is a combination

In [28]:
print(len(docs))

4


In [29]:
#Maximum marginal relevance retrieval
faiss_retriever = faiss_db.as_retriever(search_type="mmr")
docs = faiss_retriever.get_relevant_documents(query)
docs

[Document(page_content='Note: The PayFlow API also uses name-value pairs to provide parameter-based association between request and response fields of a message and their values; however, the PayFlow API is not the same as the NVP API; for more information about the PayFlow API, see the Payflow Gateway Developer Guide and Reference.\n\nPayPal API client-server architecture\n\nThe PayPal API uses a client-server model. Your website is a client of the PayPal server.\n\nA page on your website initiates an action on a PayPal API server by sending a request to the server. The PayPal server responds with a confirmation that the requested action was taken or indicates that an error occurred. The response might also contain additional information related to the request. The following diagram shows the basic request-response mechanism.', metadata={'source': 'https://developer.paypal.com/api/nvp-soap/NVPAPIOverview/'}),
 Document(page_content='To enable PayPal to authenticate your request:\n\nSp

In [30]:
faiss_retriever = faiss_db.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5}
)
docs = faiss_retriever.get_relevant_documents(query)
docs

[Document(page_content='Note: The PayFlow API also uses name-value pairs to provide parameter-based association between request and response fields of a message and their values; however, the PayFlow API is not the same as the NVP API; for more information about the PayFlow API, see the Payflow Gateway Developer Guide and Reference.\n\nPayPal API client-server architecture\n\nThe PayPal API uses a client-server model. Your website is a client of the PayPal server.\n\nA page on your website initiates an action on a PayPal API server by sending a request to the server. The PayPal server responds with a confirmation that the requested action was taken or indicates that an error occurred. The response might also contain additional information related to the request. The following diagram shows the basic request-response mechanism.', metadata={'source': 'https://developer.paypal.com/api/nvp-soap/NVPAPIOverview/'}),
 Document(page_content="Like many web services, PayPal SOAP is a combination

In [82]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain import hub
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")
combine_docs_chain = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)

faiss_retriever = faiss_db.as_retriever()
chroma_retriever = chroma_db.as_retriever()

retriever = faiss_retriever # chroma_retriever

retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [42]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
query = "how to use paypal api as client server model"
chain.invoke(query)

"To use the PayPal API in a client-server model, follow these general steps:\n\n1. **Set Up Your Client**: Your website acts as the client, from which you'll initiate PayPal API requests. This involves setting up your website's backend (server-side code) to construct and send HTTP requests to the PayPal API server.\n\n2. **Initiate an Action**: A page on your website (or server-side script) sends a request to the PayPal API server to initiate an action. This involves creating a request with the appropriate parameters and endpoints.\n\n3. **Use NVP or SOAP**: Depending on your preference or requirements, you can choose to use PayPal's Name-Value Pair (NVP) API or Simple Object Access Protocol (SOAP) API. Both APIs require different methods of request construction:\n   - For NVP, you would construct an HTTPS POST request with name-value pairs as the request parameters.\n   - For SOAP, you would generate business-object interfaces and network stubs from PayPal-provided Web Services Descri

In [35]:
from operator import itemgetter
from langchain.schema import format_document
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_core.runnables import RunnableParallel
from langchain.prompts.prompt import PromptTemplate
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [50]:
CONDENSE_QUESTION_PROMPT

PromptTemplate(input_variables=['chat_history', 'question'], template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:')

In [36]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [37]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")


def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [83]:
system_prompt_template = "You are paypal code migration assistant that converts paypal NVP code to Orders V2 REST API"
system_message_prompt = SystemMessagePromptTemplate.from_template(system_prompt_template)


# Initiate a connection to the LLM from Azure OpenAI Service via LangChain.
llm = AzureChatOpenAI(
    #system_message_prompt=system_message_prompt,
    deployment_name=os.environ["DEPLOYMENT_NAME"]
)

In [84]:
_inputs = RunnableParallel(
    standalone_question=RunnablePassthrough.assign(
        chat_history=lambda x: get_buffer_string(x["chat_history"])
    )
)
_context = {
    "context": itemgetter("standalone_question") | faiss_retriever | _combine_documents,
    "question": lambda x: x["standalone_question"],
}
conversational_qa_chain = _inputs | _context | ANSWER_PROMPT | llm

In [ ]:
query = "create helloworld java program that uses paypal NVP api for checkout using app name and client id"
conversational_qa_chain.invoke(
    {
        "question": query,
        "chat_history": [],
    }, 
)

In [85]:
java_code = """
import com.paypal.sdk.core.nvp.*;
import com.paypal.sdk.services.NVPCallerServices;

public class Checkout {

    public static void main(String[] args) {
        try {
            NVPCallerServices caller = new NVPCallerServices();
            NVPProfile profile = new NVPProfile();
            
            // Set up your API credentials and environment
            profile.setAPIUsername("YOUR_API_USERNAME");
            profile.setAPIPassword("YOUR_API_PASSWORD");
            profile.setSignature("YOUR_API_SIGNATURE");
            profile.setEnvironment("sandbox"); // Use "live" for production
            
            caller.setProfile(profile);
            
            // Create a new API request
            NVPDecoder decoder = new NVPDecoder();
            NVPEncoder encoder = new NVPEncoder();
            encoder.add("METHOD", "SetExpressCheckout");
            encoder.add("VERSION", "204"); // Specify the version of the API you're using
            encoder.add("PAYMENTREQUEST_0_AMT", "10.00"); // Set checkout amount
            encoder.add("PAYMENTREQUEST_0_CURRENCYCODE", "USD"); // Set currency
            encoder.add("RETURNURL", "http://yourwebsite.com/success"); // Set return URL after payment
            encoder.add("CANCELURL", "http://yourwebsite.com/cancel"); // Set cancel URL
            
            // Execute the API operation
            String strNVPRequest = encoder.encode();
            String strNVPResponse = caller.call(strNVPRequest);
            
            // Decode the response
            decoder.decode(strNVPResponse);
            
            if ("Success".equalsIgnoreCase(decoder.get("ACK"))) {
                System.out.println("Checkout initiated successfully.");
                // Extract token and redirect user for payment approval
                String token = decoder.get("TOKEN");
                // Redirect to PayPal for approval
                String redirectURL = "https://www.sandbox.paypal.com/cgi-bin/webscr?cmd=_express-checkout&token=" + token;
                System.out.println("Redirect user to: " + redirectURL);
            } else {
                System.out.println("Error: " + decoder.get("L_LONGMESSAGE0")); // Print error message
            }
            
        } catch (Exception e) {
            e.printStackTrace();
        }
    }
}

"""


query = f"convert the following java code to use paypal rest api instead of NVP api. full java code here: {java_code}"
conversational_qa_chain.invoke(
    {
        "question": query,
        "chat_history": [],
    }
    
)

TypeError: expected string or buffer

In [86]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

In [87]:
# First we add a step to load memory
# This adds a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)
# Now we calculate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | llm
    | StrOutputParser(),
}
# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | llm,
    "docs": itemgetter("docs"),
}
# And now we put it all together!
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [88]:
inputs = {"question": query}
result = final_chain.invoke(inputs)
result

{'answer': AIMessage(content="To implement the PayPal API in a client-server model, you can follow these steps:\n\n1. **Set Up Your Environment:**\n   - Create a PayPal developer account if you don't already have one.\n   - Log in to the PayPal Developer Dashboard and create sandbox accounts for testing. These accounts will simulate the buyer and seller in a transaction.\n   - Obtain your API credentials, which will include an API username, password, and signature or certificate for both the sandbox and live environments.\n\n2. **Choose the Right Server Endpoints:**\n   - For testing and development, use the sandbox server endpoints:\n     - With API signature: `https://api-3t.sandbox.paypal.com/nvp`\n     - With API certificate: `https://api.sandbox.paypal.com/nvp`\n   - For production, use the live server endpoints:\n     - With API signature: `https://api-3t.paypal.com/nvp`\n     - With API certificate: `https://api.paypal.com/nvp`\n\n3. **Implement Server-to-Server Communication:**

In [89]:
memory.save_context(inputs, {"answer": result["answer"].content})

In [90]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='how to use paypal api as client server model'),
  AIMessage(content="To implement the PayPal API in a client-server model, you can follow these steps:\n\n1. **Set Up Your Environment:**\n   - Create a PayPal developer account if you don't already have one.\n   - Log in to the PayPal Developer Dashboard and create sandbox accounts for testing. These accounts will simulate the buyer and seller in a transaction.\n   - Obtain your API credentials, which will include an API username, password, and signature or certificate for both the sandbox and live environments.\n\n2. **Choose the Right Server Endpoints:**\n   - For testing and development, use the sandbox server endpoints:\n     - With API signature: `https://api-3t.sandbox.paypal.com/nvp`\n     - With API certificate: `https://api.sandbox.paypal.com/nvp`\n   - For production, use the live server endpoints:\n     - With API signature: `https://api-3t.paypal.com/nvp`\n     - With API certificate: `https

In [91]:
inputs = {"question": "but what is Name-Value Pair API "}
result = final_chain.invoke(inputs)
result

{'answer': AIMessage(content="A Name-Value Pair (NVP) API is an interface provided by PayPal that allows communication between a client website and the PayPal server using parameter-based associations. This API operates by sending request messages with specific parameters and their associated values, and then receiving response messages from PayPal with corresponding values. The format of these messages is structured with names and values in strings, where names are separated from values by an equal sign (=), and individual name-value pairs are separated by an ampersand (&). Each value in the NVP string is also URL-encoded. The NVP API is used to execute various PayPal API operations by specifying the operation name, version, and additional information as needed in the request. The response from the PayPal server will include common fields as well as any information specific to the requested operation, such as the customer's shipping address in the case of a successful request for cust